# Get annotations from Trinotate report to modify PASA gff3 file

In [1]:
tab1 = open('~/WRASSE/pasaA_annotation/pasaA_trinotate_annotation_report.xls', 'r')

In [2]:
# dict of dicts
zfish = {}
tilapia = {}
zlist = []
tlist = []
sprot = {}
pfam = {}
for line in tab1:
    parts = line.split('\t')
    tran = parts[1]
    sp = parts[2]
    
    zp = parts[9]
    if zp.startswith('E'):
        zpC = zp.split('^')[0]
        zfish[tran]=zpC
        zlist.append(zpC)
    tp = parts[10]
    if tp.startswith('E'):
        tpC = tp.split('^')[0]
        tilapia[tran]=tpC
        tlist.append(tpC)
    pf = parts[11]
    if pf.startswith('P'):
        pfam[tran]=pf
    if sp.startswith('.'):
        continue
    else:
        spC = sp.split('^')[0]
        sprot[tran]=spC
    
    
    
    
    

print(len(zfish))
print(len(sprot))
tab1.close()

17051
24422


In [5]:
zfish['asmbl_34']

'ENSDARP00000086301.5'

In [6]:
# output zlist and tlist to file
zout = open('zebrafish_peptides_from_trinotateA.txt','w')
tout = open('tilipia_peptides_from_trinotateA.txt','w')


In [7]:
for z in zlist:
    zout.write(z+'\n')
zout.close()

In [8]:
for t in tlist:
    tout.write(t+'\n')
tout.close()

## Output zebrafish gene names and descriptions

In [10]:
# first import the descriptions
descript_file = open('~/WRASSE/zebrafishA_peptide_gene-name_and_descriptions.txt')

In [11]:
descripts = {}
geneNames = {} # adding gene name dict for searching later
for line in descript_file:
    if line.startswith('ensembl'):
        continue
    else:
        line = line.strip('\n')
        parts = line.split('\t')
        ensembl = parts[0]
        desc = parts[3]
        des1 = desc.split(' [')[0]
        des = des1.replace(' ','_').replace(':','-').replace('.','-').replace(';','_').replace(',','_').replace("'","")
        descripts[ensembl]=des
        genename = parts[2]
        geneNames[ensembl]=genename
descript_file.close()

In [13]:
descripts['ENSDARP00000006201.7']

'phosphoglycerate_mutase_1a'

In [14]:
geneNames['ENSDARP00000006201.7']

'pgam1a'

In [15]:
gff = open('~/WRASSE/pasaA_annotation/bluePasa_1A.assemblies.fasta.transdecoder.genome.gff3')

In [16]:
newgff = open('ann_blue_genemaps_with_zebrafish_annotations.gff3','w')

In [17]:
# try with modified format, and changing CDS
for line in gff:
    if line.startswith('\n'):
        continue
    else:
        line = line.strip('\n')
        parts = line.split('\t')
        genetype = parts[2]
        description = parts[8]
        if genetype == 'CDS':
            assembly = description.split('|')[0][7:]
        else:
            assembly = description.split('|')[0][3:]
        if assembly in zfish:
            pepcode = zfish[assembly]
            # now add description
            if pepcode in descripts:
                pep_desc = descripts[pepcode]
                gene_name = geneNames[pepcode]
                desc_parts = description.split(';')
                fullID = desc_parts[0]
                id_parts =  fullID.split('|')
                idtype = id_parts[1]
                idname = id_parts[0]
                new_idname = idname.replace(assembly,pep_desc).replace('ID=cds.','ID=')
                if genetype == 'CDS':
                    new_idtype = idtype+'.cds'
                else:
                    new_idtype = idtype
                # change the subsequent part
                parent = desc_parts[1]
                newparent = parent.replace(assembly,pep_desc).replace('ORF',gene_name)
                if genetype == 'gene':
                    
                    new_description = new_idname + '|' + new_idtype + ';' + newparent
                elif genetype == 'mRNA':
                    new_description = new_idname + '|' + new_idtype + ';' + newparent
                else:
                    new_description = new_idname + '|' + new_idtype + ';' + newparent + ';Name='+gene_name
                newline = line.replace(description,new_description) #(assembly,pep_desc).replace('ORF',pepcode)
                newgff.write(newline+'\n')
newgff.close()
gff.close()

## Also produce separate file that has sprot or no annotation

In [18]:
gff = open('~/WRASSE/pasaA_annotation/bluePasa_1A.assemblies.fasta.transdecoder.genome.gff3')

In [19]:
newgff = open('ann_blue_annotations.gff3','w')

In [20]:
# try with modified format, and changing CDS
for line in gff:
    if line.startswith('\n'):
        continue
    else:
        line = line.strip('\n')
        parts = line.split('\t')
        genetype = parts[2]
        description = parts[8]
        if genetype == 'CDS':
            assembly = description.split('|')[0][7:]
        else:
            assembly = description.split('|')[0][3:]
        if assembly in sprot:
            pepcode = sprot[assembly]
            # now add description
            #if pepcode in descripts:
            #pep_desc = descripts[pepcode]
            desc_parts = description.split(';')
            fullID = desc_parts[0]
            id_parts =  fullID.split('|')
            idtype = id_parts[1]
            idname = id_parts[0]
            new_idname = idname.replace(assembly,pepcode).replace('ID=cds.','ID=')
            if genetype == 'CDS':
                new_idtype = idtype+'.cds'
            else:
                new_idtype = idtype
            # change the subsequent part
            parent = desc_parts[1]
            newparent = parent.replace(assembly,pepcode)#.replace('ORF',pepcode)
            new_description = new_idname + '|' + new_idtype + ';' + newparent
            newline = line.replace(description,new_description) #(assembly,pep_desc).replace('ORF',pepcode)
            newgff.write(newline+'\n')
        else:
            newgff.write(line+'\n')
newgff.close()
gff.close()